In [ ]:
#importing the libraries
from urllib.request import urlopen
import requests 
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np
from collections import defaultdict



In [ ]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
   'sec-ch-ua-platform': '"macOS"',
   'sec-fetch-dest': 'empty',
   'sec-fetch-mode': 'cors',
   'sec-fetch-site': 'same-origin'
}


In [ ]:
exceldata = pd.read_excel("Udemy courses excel path",sheet_name = "urls")
Udemy_urls = [] 
for idx in exceldata.index:
    Udemy_url = exceldata['course url'][idx]
    Udemy_urls.append(Udemy_url)
Udemy_urls = list(set(Udemy_urls)) # removing duplicates

In [ ]:
domain_url = "https://www.udemy.com"

# Initialize lists
Instructor_url = []
Course_url = []
Instructor_name = []
Instructor_rating = []
No_of_reviews = []
No_of_students = []
No_of_courses = []
Linkedin_url = []
Youtube_url = []

count = 0

for url in Udemy_urls:
    count += 1
    print(count, url)

    try:
        # Perform initial requests and parsing
        r = requests.Session().get(url, headers=req_headers, timeout=(5, 60))
        soup = BS(r.content, 'html.parser')

        all_urls = [a['href'] for a in soup('a') if a.has_attr('href')]
        regex = r"^/user/?"

        
        user_url = [u for u in all_urls if re.match(regex, u)]
        instructor_url = domain_url + user_url[0] if user_url else "N/A"

        if instructor_url != "N/A":
            r2 = requests.Session().get(instructor_url, headers=req_headers, timeout=(5, 60))
            soup2 = BS(r2.content, 'html.parser')

            instructor_name, no_of_students, instructor_rating, no_of_reviews, no_of_courses = None, None, None, None, None

            all_urls = [a['href'] for a in soup2('a')  if a.has_attr('href')]

            regex_linked_in = r"^https://linkedin.com/?"
            linkedin_url = [u for u in all_urls if re.match(regex_linked_in, u)]

            if len(linkedin_url)!=0:
                Linkedin_url.append(linkedin_url[0])
            else:
                Linkedin_url.append("")

            regex_youtube = r"^https://www.youtube.com/?"
            youtube_url = [u for u in all_urls if re.match(regex_youtube, u)]

            if len(youtube_url)!=0:
                Youtube_url.append(youtube_url[0])
            else:
                Youtube_url.append("")


            instructor_stats = soup.find('div',attrs={'class':'instructor--instructor__image-and-stats--1MRZH'})
            instructor_name = instructor_stats.find('img')['alt']


            stats = instructor_stats.findAll('div',attrs={'class':'ud-block-list-item ud-block-list-item-small ud-block-list-item-tight ud-block-list-item-neutral ud-text-sm'}) if instructor_stats.find('div') else None

            for i in stats:

                text = i.get_text()
                if 'Instructor Rating' in text:
                    instructor_rating = text.split(' ')[0]   
                elif 'Reviews' in text:
                    no_of_reviews = text.split(' ')[0]
                elif 'Students' in text:
                    no_of_students = text.split(' ')[0]
                elif 'Courses' in text:
                    no_of_courses = text.split(' ')[0]

            if instructor_name != None:         
                Instructor_name.append(instructor_name)
            else:
                Instructor_name.append("")


            if instructor_rating != None:         
                Instructor_rating.append(instructor_rating)
            else:
                Instructor_rating.append("")


            if no_of_reviews != None:         
                No_of_reviews.append(no_of_reviews)
            else:
                No_of_reviews.append("")


            if no_of_students != None:         
                No_of_students.append(no_of_students)
            else:
                No_of_students.append("")


            if no_of_courses != None:         
                No_of_courses.append(no_of_courses)
            else:
                No_of_courses.append("")
            
            Course_url.append(url)
            Instructor_url.append(instructor_url)

        else:
            Course_url.append(url)
            Instructor_url.append(instructor_url)
            Linkedin_url.append("N/A")
            Youtube_url.append("N/A")
            Instructor_name.append("N/A") 
            Instructor_rating.append("N/A")
            No_of_reviews.append("N/A")
            No_of_students.append("N/A")
            No_of_courses.append("N/A")

    except requests.exceptions.Timeout:
        print("Timeout occurred")
        # Handle timeout error here if needed

# Ensure all lists have the same length before further processing
assert (
    len(Instructor_url)
    == len(Course_url)
    == len(Instructor_name)
    == len(Instructor_rating)
    == len(No_of_reviews)
    == len(No_of_students)
    == len(No_of_courses)
    == len(Linkedin_url)
    == len(Youtube_url)
), "All arrays must have the same length"


In [ ]:
df = pd.DataFrame(data = {'Udemy course URL':Course_url,
                          'Instructor URL':Instructor_url,
                          'Intructor name':Instructor_name,
                          'Instructor rating':Instructor_rating,
                          'Number of reviews':No_of_reviews,
                          'Number of students':No_of_students,
                          'Number of courses':No_of_courses,
                          'LinkedIN URL': Linkedin_url,
                          'Youtube URL': Youtube_url})

In [ ]:
df.to_csv('Scraped_udemy_data_2.csv')

In [ ]:
NAfiltered_df = df[df["Instructor URL"] != "N/A"]
condition_2 = (NAfiltered_df["LinkedIN URL"] != "") & (NAfiltered_df["Youtube URL"] != "")
filtered_df = NAfiltered_df[condition_2]
filtered_df

In [ ]:
filtered_df.to_csv('Scraped_filtered_data_2.csv')